# Clustering the aspects of the reviews

## Loading the reviews from the database

In [1]:
%cd ..
%cd ..

/home/pdona/VariousExercises/SentimentAnalysisAmazonMerchant/SentimentAnalysis
/home/pdona/VariousExercises/SentimentAnalysisAmazonMerchant


Import the classes and database ORM

In [2]:
import ProductReviewScraper.amazon_review_database as db
from ProductReviewScraper.models import Review, Product
import pandas as pd

In [3]:
df = pd.read_sql(
    db.session.query(
        Product, Review).filter(Review.product_id == Product.id).statement,
    con=db.engine
)

df.drop(['id', 'id_1', 'product_id'],
        axis='columns', inplace=True)
df.columns = ['product_asin', 'product_name',  'product_imageurl',
              'product_price', 'product_global_rating', 'review_amazonid',
              'review_user', 'review_rating', 'review_title',
              'review_date', 'review_verified', 'review_foreign',
              'review_content', 'review_helpful_vote']
dftmp = df.copy()
df = None

We use only reviews written in english

In [4]:
df = dftmp[dftmp["review_foreign"] == False].copy()
df.drop(['review_foreign'],
        axis='columns', inplace=True)


There is a strange cap at 5K reviews - To investigate.

We cleanup the unicode in the reviews

In [5]:
import unidecode

In [6]:
df["review_content"] = df["review_content"].apply(unidecode.unidecode)
df["review_title"] = df["review_title"].apply(unidecode.unidecode)
df["product_name"] = df["product_name"].apply(unidecode.unidecode)

In this notebook we focus on only one product

In [7]:
dfproduct = df.query("product_asin=='B000TVJ6XW'").copy()


## Perform aspect extraction

In [ ]:
from SentimentAnalysis import aspect_estraction

We extract the aspects and keep track of the IDs to add the review features the the aspect

In [ ]:
aspects_pool = []
ids = []

for review, reviewID in zip(list(dfproduct["review_content"]), list(dfproduct.index)):
    tmp = aspect_estraction.get_aspects(review)
    aspects_pool += tmp
    ids += [reviewID]*len(tmp)

To cluster the aspects we use the spacy vector representation of the aspect words

In [ ]:
aspects_words = [aspect_word.aspect for aspect_word in aspects_pool]
words_polarity = [aspect_word.sentiment for aspect_word in aspects_pool]

aspect_vectors = [aspect_estraction.nlp(
    aspect).vector for aspect in aspects_words]

We use a basic KMeans algorithm to cluster the reviews. Even after aspects extraction there are too many aspects to derive a business insight. We cluster the aspects to group them into similar themes (aka group scent, smell, aroma, etc for candle related reviews)

In [ ]:
# Load all the reviews
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5,random_state=42)
kmeans.fit(aspect_vectors)
labels = kmeans.labels_

aspectsdf = pd.DataFrame(
    [z for z in zip(aspects_words, labels, words_polarity)], columns=["aspect", "cluster", "polarity"])
aspectsdf.index = ids
aspectsdfclustered = pd.merge(aspectsdf, df, left_index=True, right_index=True)


We visualize the 5 clusters

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(data=aspectsdfclustered, x="review_rating",hue="cluster")


In [ ]:
fig, axis = plt.subplots(3, 2, figsize=(20, 15))
sns.countplot(data=aspectsdfclustered.query(
    "cluster==0"), x="review_rating", ax=axis[0, 0], color=sns.color_palette()[0])
sns.countplot(data=aspectsdfclustered.query(
    "cluster==1"), x="review_rating", ax=axis[0, 1], color=sns.color_palette()[1])
sns.countplot(data=aspectsdfclustered.query(
    "cluster==2"), x="review_rating", ax=axis[1, 0], color=sns.color_palette()[2])
sns.countplot(data=aspectsdfclustered.query(
    "cluster==3"), x="review_rating", ax=axis[1, 1], color=sns.color_palette()[3])
sns.countplot(data=aspectsdfclustered.query(
    "cluster==4"), x="review_rating", ax=axis[2, 0], color=sns.color_palette()[4])


The rating distribution looks similar for different clusters, however cluster 0 and 3 have an excess of negative reviews.

In [ ]:
aspectsdfclustered.columns

How to name clusters

In [ ]:
testcluster = aspectsdfclustered.query("cluster==0").copy()
name=aspectsdfclustered.query("cluster==0")[
    'aspect'].value_counts().first_valid_index()
print(name)
testcluster['aspect'].value_counts().nlargest(5)

In [ ]:
testcluster = aspectsdfclustered.query("cluster==1").copy()
name = aspectsdfclustered.query("cluster==1")[
    'aspect'].value_counts().first_valid_index()
print(name)
testcluster['aspect'].value_counts().nlargest(5)


In [ ]:
testcluster = aspectsdfclustered.query("cluster==2").copy()
name = aspectsdfclustered.query("cluster==2")[
    'aspect'].value_counts().first_valid_index()
print(name)
testcluster['aspect'].value_counts().nlargest(5)


In [ ]:
testcluster = aspectsdfclustered.query("cluster==3").copy()
name = aspectsdfclustered.query("cluster==3")[
    'aspect'].value_counts().first_valid_index()
print(name)
testcluster['aspect'].value_counts(normalize=True).nlargest(5)


In [ ]:
testcluster = aspectsdfclustered.query("cluster==4").copy()
name = aspectsdfclustered.query("cluster==4")[
    'aspect'].value_counts().first_valid_index()
print(name)
testcluster['aspect'].value_counts().nlargest(5)

In [ ]:
cluster_names = {
    0: "Emotions", 
    1: "Scent",
    2: "Material", 
    3: "Price", 
    4: "Duration", 
    }

In [ ]:
aspectsdfclustered["cluster_label"] = aspectsdfclustered["cluster"].map(cluster_names)

In [ ]:
aspectsdfclustered["cluster_label"]

In [8]:
from SentimentAnalysis.aspect_clustering import extract_aspects

2022-03-02 18:13:51.165005: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-02 18:13:51.165023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
aspectsdfclustered = extract_aspects(dfproduct, 'B000TVJ6XW')


In [24]:
pospolarity = aspectsdfclustered.query("polarity>0")["cluster"].value_counts()
neupolarity = aspectsdfclustered.query("polarity==0")["cluster"].value_counts()
negpolarity = aspectsdfclustered.query("polarity<0")["cluster"].value_counts()
posrating = aspectsdfclustered.query("review_rating>3")["cluster"].value_counts()
neurating = aspectsdfclustered.query("review_rating==3")["cluster"].value_counts()
negrating = aspectsdfclustered.query("review_rating<3")["cluster"].value_counts()


In [25]:
polaritycount = pd.DataFrame({
    "Cluster": pospolarity.index,
    "PositivePolarity": pospolarity.values,
    "NeutralPolarity": neupolarity.values,
    "NegativePolarity": negpolarity.values,
    "PositiveRating": posrating.values,
    "NeutralRating": neurating.values,
    "NegativeRating": negrating.values})
